# Step 6: RNN (Recurrent Neural Network) - 순차 데이터 처리

RNN은 순차적인 데이터(텍스트, 시계열 등)를 처리하는 데 특화된 신경망입니다. 이전 정보를 기억하여 현재 예측에 활용합니다.

## 학습 목표
1. RNN의 기본 원리 이해
2. Vanilla RNN, LSTM, GRU 구현
3. 텍스트 생성 모델 만들기
4. 감성 분석 수행하기
5. 시계열 예측하기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import string
import random
from collections import Counter
import pandas as pd
from tqdm import tqdm

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# 시드 설정
torch.manual_seed(42)
np.random.seed(42)

## 1. RNN의 기본 개념

RNN은 시간 단계별로 정보를 처리하며, 이전 상태를 다음 단계로 전달합니다.

### 1.1 Vanilla RNN 직접 구현

In [ ]:
class VanillaRNNCell(nn.Module):
    """단일 RNN 셀 구현"""
    def __init__(self, input_size, hidden_size):
        super(VanillaRNNCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # 가중치 초기화
        self.i2h = nn.Linear(input_size, hidden_size)  # 입력 → 은닉
        self.h2h = nn.Linear(hidden_size, hidden_size) # 은닉 → 은닉
        self.tanh = nn.Tanh()
    
    def forward(self, x, hidden):
        # h_t = tanh(W_ih * x_t + W_hh * h_{t-1} + b)
        new_hidden = self.tanh(self.i2h(x) + self.h2h(hidden))
        return new_hidden

# RNN 셀 테스트
rnn_cell = VanillaRNNCell(input_size=10, hidden_size=20)

# 입력: (batch_size, input_size)
x = torch.randn(32, 10)
h = torch.randn(32, 20)

new_h = rnn_cell(x, h)
print(f"입력 형태: {x.shape}")
print(f"이전 은닉 상태: {h.shape}")
print(f"새로운 은닉 상태: {new_h.shape}")

### 1.2 전체 RNN 레이어 구현

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # PyTorch의 RNN 레이어 사용
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, x, h0=None):
        # x: (batch, seq_len, input_size)
        batch_size = x.size(0)
        
        # 초기 은닉 상태
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        # RNN forward
        out, hn = self.rnn(x, h0)
        # out: (batch, seq_len, hidden_size)
        # hn: (num_layers, batch, hidden_size)
        
        return out, hn

# RNN 테스트
rnn = SimpleRNN(input_size=10, hidden_size=20, num_layers=2)

# 시퀀스 입력: (batch_size, seq_len, input_size)
seq_input = torch.randn(32, 15, 10)  # 32개 배치, 15 시간 단계, 10차원 입력

output, hidden = rnn(seq_input)
print(f"입력 시퀀스 형태: {seq_input.shape}")
print(f"출력 형태: {output.shape}")
print(f"최종 은닉 상태 형태: {hidden.shape}")

## 2. LSTM과 GRU

기본 RNN의 장기 의존성 문제를 해결하기 위한 개선된 구조들입니다.

In [ ]:
# RNN 변형들의 구조 비교
class RNNComparison(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNNComparison, self).__init__()
        
        # 세 가지 RNN 타입
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
    
    def forward(self, x):
        # RNN
        rnn_out, _ = self.rnn(x)
        
        # LSTM
        lstm_out, (lstm_h, lstm_c) = self.lstm(x)
        
        # GRU
        gru_out, _ = self.gru(x)
        
        return rnn_out, lstm_out, gru_out

# 모델 생성 및 파라미터 수 비교
comparison_model = RNNComparison(input_size=10, hidden_size=20)

print("파라미터 수 비교:")
print(f"RNN:  {sum(p.numel() for p in comparison_model.rnn.parameters()):,}")
print(f"LSTM: {sum(p.numel() for p in comparison_model.lstm.parameters()):,}")
print(f"GRU:  {sum(p.numel() for p in comparison_model.gru.parameters()):,}")

# LSTM의 게이트 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# RNN
axes[0].text(0.5, 0.5, 'RNN\n\nh_t = tanh(W_ih·x_t + W_hh·h_{t-1})', 
             ha='center', va='center', fontsize=12, transform=axes[0].transAxes)
axes[0].set_title('Vanilla RNN')
axes[0].axis('off')

# LSTM
axes[1].text(0.5, 0.7, 'LSTM', ha='center', va='center', fontsize=14, 
             weight='bold', transform=axes[1].transAxes)
axes[1].text(0.5, 0.5, 'Forget Gate: f_t\nInput Gate: i_t\nOutput Gate: o_t\nCell State: c_t', 
             ha='center', va='center', fontsize=10, transform=axes[1].transAxes)
axes[1].set_title('Long Short-Term Memory')
axes[1].axis('off')

# GRU
axes[2].text(0.5, 0.6, 'GRU', ha='center', va='center', fontsize=14, 
             weight='bold', transform=axes[2].transAxes)
axes[2].text(0.5, 0.4, 'Reset Gate: r_t\nUpdate Gate: z_t', 
             ha='center', va='center', fontsize=10, transform=axes[2].transAxes)
axes[2].set_title('Gated Recurrent Unit')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 3. 문자 단위 텍스트 생성

RNN을 사용하여 텍스트를 학습하고 생성해봅시다.

In [ ]:
# 간단한 텍스트 데이터
text = """Deep learning is a subset of machine learning in artificial intelligence.
It has networks capable of learning unsupervised from data that is unstructured.
Deep learning algorithms are similar to how nervous system structured.
The deep learning models are trained by using large sets of labeled data."""

# 문자 집합 생성
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

print(f"텍스트 길이: {len(text)}")
print(f"고유 문자 수: {vocab_size}")
print(f"문자 집합: {''.join(chars)}")

# 텍스트를 인덱스로 변환
data = [char_to_idx[ch] for ch in text]
print(f"\n인코딩 예시: '{text[:20]}' → {data[:20]}")

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=2):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        
        # LSTM 레이어
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, 
                           batch_first=True, dropout=0.2 if num_layers > 1 else 0)
        
        # 출력 레이어
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, hidden=None):
        # x: (batch, seq_len)
        embed = self.embedding(x)  # (batch, seq_len, hidden_size)
        
        # LSTM forward
        out, hidden = self.lstm(embed, hidden)
        
        # 출력 변환
        out = self.fc(out)  # (batch, seq_len, vocab_size)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        return (weight.new_zeros(self.num_layers, batch_size, self.hidden_size),
                weight.new_zeros(self.num_layers, batch_size, self.hidden_size))

# 모델 생성
char_rnn = CharRNN(vocab_size, hidden_size=128, num_layers=2).to(device)
print("Character RNN 모델:")
print(char_rnn)

### 3.1 텍스트 생성을 위한 학습

In [ ]:
# 학습 데이터 준비
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+1:i+seq_length+1]
        sequences.append(seq)
        targets.append(target)
    
    return torch.tensor(sequences), torch.tensor(targets)

# 시퀀스 생성
seq_length = 40
X, y = create_sequences(data, seq_length)
print(f"학습 시퀀스 수: {len(X)}")
print(f"시퀀스 형태: {X.shape}, 타겟 형태: {y.shape}")

# 데이터로더
dataset = torch.utils.data.TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 학습
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(char_rnn.parameters(), lr=0.001)

# 학습 함수
def train_char_rnn(model, dataloader, epochs=50):
    model.train()
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        hidden = model.init_hidden(32)
        
        for batch_x, batch_y in dataloader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Hidden state detach
            hidden = tuple([h.detach() for h in hidden])
            
            # Forward
            output, hidden = model(batch_x, hidden)
            loss = criterion(output.view(-1, vocab_size), batch_y.view(-1))
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
    
    return losses

# 학습 실행
losses = train_char_rnn(char_rnn, dataloader, epochs=100)

In [ ]:
# 텍스트 생성 함수
def generate_text(model, start_string, length=100, temperature=1.0):
    model.eval()
    
    # 시작 문자열을 인덱스로 변환
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = torch.tensor(input_eval).unsqueeze(0).to(device)
    
    # 생성된 텍스트 저장
    text_generated = list(start_string)
    
    # 은닉 상태 초기화
    hidden = model.init_hidden(1)
    
    with torch.no_grad():
        for i in range(length):
            # 예측
            output, hidden = model(input_eval, hidden)
            
            # 마지막 문자의 출력 사용
            output = output[:, -1, :] / temperature
            probabilities = F.softmax(output, dim=-1)
            
            # 샘플링
            predicted_id = torch.multinomial(probabilities, 1).item()
            
            # 다음 입력 준비
            input_eval = torch.tensor([[predicted_id]]).to(device)
            
            # 생성된 문자 추가
            text_generated.append(idx_to_char[predicted_id])
    
    return ''.join(text_generated)

# 다양한 온도로 텍스트 생성
print("생성된 텍스트:\n")
for temp in [0.5, 0.8, 1.0, 1.2]:
    print(f"\nTemperature = {temp}:")
    generated = generate_text(char_rnn, "Deep learning ", length=100, temperature=temp)
    print(generated)

## 4. 감성 분석 (Sentiment Analysis)

영화 리뷰의 긍정/부정을 분류하는 모델을 만들어봅시다.

In [ ]:
# 간단한 감성 분석 데이터셋
reviews = [
    ("This movie is fantastic! Best film I've seen all year.", 1),
    ("Terrible movie. Complete waste of time.", 0),
    ("Amazing cinematography and great acting.", 1),
    ("Boring plot and poor character development.", 0),
    ("I loved every minute of this film!", 1),
    ("One of the worst movies ever made.", 0),
    ("Brilliant storytelling and excellent direction.", 1),
    ("Fell asleep halfway through. Very disappointing.", 0),
    ("A masterpiece! Highly recommend to everyone.", 1),
    ("Predictable and unoriginal. Not worth watching.", 0),
]

# 텍스트 전처리
def preprocess_text(text):
    # 소문자 변환 및 구두점 제거
    text = text.lower()
    text = ''.join([c for c in text if c not in string.punctuation])
    return text.split()

# 어휘 사전 구축
all_words = []
for review, _ in reviews:
    all_words.extend(preprocess_text(review))

word_counts = Counter(all_words)
vocab = ['<PAD>', '<UNK>'] + [word for word, _ in word_counts.most_common()]
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for idx, word in enumerate(vocab)}

print(f"어휘 크기: {len(vocab)}")
print(f"가장 빈번한 단어: {word_counts.most_common(10)}")

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=2, bidirectional=True, dropout=0.5):
        super(SentimentRNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                           bidirectional=bidirectional, dropout=dropout, 
                           batch_first=True)
        
        # 양방향 LSTM인 경우 hidden_dim * 2
        fc_input_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, text, text_lengths):
        # text: (batch, seq_len)
        embedded = self.embedding(text)  # (batch, seq_len, embedding_dim)
        
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, 
                                                           batch_first=True, 
                                                           enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        # 마지막 은닉 상태 사용
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        
        # 분류
        output = self.fc(hidden)
        
        return output

# 모델 생성
sentiment_model = SentimentRNN(
    vocab_size=len(vocab),
    embedding_dim=100,
    hidden_dim=256,
    output_dim=1,
    n_layers=2,
    bidirectional=True,
    dropout=0.5
).to(device)

print("감성 분석 모델:")
print(sentiment_model)

## 5. 시계열 예측

RNN을 사용하여 시계열 데이터를 예측해봅시다.

In [ ]:
# 사인파 데이터 생성
def create_sine_wave_data(seq_length=50, num_samples=1000):
    x = np.linspace(0, 100, num_samples)
    y = np.sin(x) + 0.1 * np.random.randn(num_samples)  # 노이즈 추가
    
    sequences = []
    targets = []
    
    for i in range(len(y) - seq_length):
        seq = y[i:i+seq_length]
        target = y[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

# 데이터 생성
X_time, y_time = create_sine_wave_data()
print(f"시계열 데이터 형태: {X_time.shape}")
print(f"타겟 형태: {y_time.shape}")

# 학습/테스트 분할
split_idx = int(0.8 * len(X_time))
X_train = torch.FloatTensor(X_time[:split_idx]).unsqueeze(-1)
y_train = torch.FloatTensor(y_time[:split_idx])
X_test = torch.FloatTensor(X_time[split_idx:]).unsqueeze(-1)
y_test = torch.FloatTensor(y_time[split_idx:])

# 시각화
plt.figure(figsize=(12, 4))
plt.plot(y_time[:200], label='Sine Wave with Noise')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
class TimeSeriesRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super(TimeSeriesRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # GRU 사용
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         batch_first=True, dropout=0.2 if num_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, _ = self.gru(x)
        
        # 마지막 시간 단계의 출력 사용
        out = self.fc(out[:, -1, :])
        
        return out

# 모델 생성 및 학습
ts_model = TimeSeriesRNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(ts_model.parameters(), lr=0.001)

# 데이터로더
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 학습
ts_model.train()
losses = []

for epoch in range(100):
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        output = ts_model(batch_x)
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {avg_loss:.4f}')

In [ ]:
# 예측 및 시각화
ts_model.eval()
with torch.no_grad():
    test_pred = ts_model(X_test.to(device)).cpu().numpy()

# 결과 시각화
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# 학습 손실
axes[0].plot(losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# 예측 결과
axes[1].plot(y_test[:100], label='Actual', alpha=0.7)
axes[1].plot(test_pred[:100], label='Predicted', alpha=0.7)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Value')
axes[1].set_title('Time Series Prediction')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 예측 성능
mse = np.mean((y_test.numpy() - test_pred.squeeze())**2)
print(f"\nTest MSE: {mse:.4f}")

## 6. Attention 메커니즘

시퀀스의 중요한 부분에 집중하는 어텐션 메커니즘을 구현해봅시다.

In [ ]:
class AttentionRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # Attention 가중치 계산
        self.attention = nn.Linear(hidden_size, 1)
        
        # 출력 레이어
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # LSTM 출력
        lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden_size)
        
        # Attention 점수 계산
        attention_scores = self.attention(lstm_out)  # (batch, seq_len, 1)
        attention_weights = F.softmax(attention_scores, dim=1)  # (batch, seq_len, 1)
        
        # 가중 평균
        weighted = lstm_out * attention_weights  # (batch, seq_len, hidden_size)
        context = torch.sum(weighted, dim=1)  # (batch, hidden_size)
        
        # 출력
        output = self.fc(context)
        
        return output, attention_weights

# Attention 시각화
def visualize_attention(text, attention_weights):
    fig, ax = plt.subplots(figsize=(10, 2))
    
    # 히트맵
    im = ax.imshow(attention_weights.T, cmap='Blues', aspect='auto')
    
    # 텍스트 라벨
    ax.set_xticks(range(len(text)))
    ax.set_xticklabels(text, rotation=45)
    ax.set_yticks([0])
    ax.set_yticklabels(['Attention'])
    
    # 컬러바
    plt.colorbar(im, ax=ax)
    plt.title('Attention Weights')
    plt.tight_layout()
    plt.show()

# 예시 실행
attention_model = AttentionRNN(input_size=10, hidden_size=32, output_size=2)
sample_input = torch.randn(1, 10, 10)  # (batch=1, seq_len=10, input_size=10)
output, attn_weights = attention_model(sample_input)

print(f"출력 형태: {output.shape}")
print(f"Attention 가중치 형태: {attn_weights.shape}")

# Attention 가중치 시각화
sample_text = ['The', 'movie', 'was', 'really', 'good', 'and', 'I', 'enjoyed', 'it', '!']
visualize_attention(sample_text, attn_weights[0].detach().numpy())

## 7. 양방향 RNN과 다층 구조

In [ ]:
# 다양한 RNN 구조 비교
class RNNArchitectures(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNArchitectures, self).__init__()
        
        # 1. 단방향 단층 RNN
        self.simple_rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        
        # 2. 양방향 RNN
        self.bidirectional_rnn = nn.LSTM(input_size, hidden_size, 
                                        batch_first=True, bidirectional=True)
        
        # 3. 다층 RNN
        self.multilayer_rnn = nn.LSTM(input_size, hidden_size, 
                                     num_layers=3, batch_first=True, dropout=0.2)
        
        # 4. 양방향 다층 RNN
        self.bidirectional_multilayer = nn.LSTM(input_size, hidden_size, 
                                               num_layers=3, batch_first=True, 
                                               bidirectional=True, dropout=0.2)
    
    def forward(self, x):
        # 각 구조의 출력
        simple_out, _ = self.simple_rnn(x)
        bi_out, _ = self.bidirectional_rnn(x)
        multi_out, _ = self.multilayer_rnn(x)
        bi_multi_out, _ = self.bidirectional_multilayer(x)
        
        return simple_out, bi_out, multi_out, bi_multi_out

# 구조 비교
architectures = RNNArchitectures(input_size=10, hidden_size=20, output_size=5)
sample_input = torch.randn(2, 15, 10)  # (batch=2, seq_len=15, input_size=10)

outputs = architectures(sample_input)
print("다양한 RNN 구조의 출력 형태:")
print(f"단방향 단층: {outputs[0].shape}")
print(f"양방향 단층: {outputs[1].shape}")
print(f"단방향 다층: {outputs[2].shape}")
print(f"양방향 다층: {outputs[3].shape}")

# 파라미터 수 비교
print("\n파라미터 수:")
for name, module in architectures.named_children():
    param_count = sum(p.numel() for p in module.parameters())
    print(f"{name}: {param_count:,}")

## 8. 연습 문제

In [ ]:
# 문제 1: Sequence-to-Sequence 모델 구현
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        # 힌트: Encoder와 Decoder 두 개의 RNN 필요
        # Encoder: 입력 시퀀스를 고정 크기 벡터로 인코딩
        # Decoder: 인코딩된 벡터를 출력 시퀀스로 디코딩
        pass

# 문제 2: 단어 단위 언어 모델 구현
class WordLevelLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(WordLevelLM, self).__init__()
        # 힌트: 
        # 1. 단어 임베딩
        # 2. LSTM 레이어
        # 3. 다음 단어 예측을 위한 출력층
        pass

# 문제 3: 다변량 시계열 예측
# 여러 특징을 가진 시계열 데이터 처리
def create_multivariate_data():
    # 힌트: 여러 개의 관련된 시계열 생성
    # 예: 온도, 습도, 기압 등
    pass

## 정리

이번 튜토리얼에서 배운 내용:
1. RNN의 기본 원리와 구현
2. LSTM과 GRU의 구조와 장점
3. 문자 단위 텍스트 생성
4. 감성 분석을 위한 텍스트 분류
5. 시계열 데이터 예측
6. Attention 메커니즘
7. 양방향 및 다층 RNN 구조

### RNN의 핵심 개념:
- **순차적 정보 처리**: 이전 정보를 기억하여 현재 예측에 활용
- **가변 길이 입력**: 다양한 길이의 시퀀스 처리 가능
- **장기 의존성**: LSTM/GRU로 긴 시퀀스의 정보 보존
- **양방향 처리**: 과거와 미래 정보 모두 활용

축하합니다! 이제 딥러닝의 기초부터 CNN, RNN까지 모두 학습했습니다. 
다음 단계로는 Transformer, GAN, 강화학습 등 더 고급 주제들을 탐구해보세요!